In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from dataloader import get_evaluation_datasets_by_client  # Assuming this function gets local client datasets
from model import Net
from collections import OrderedDict
from config import NUM_CLASSES, NUM_CLIENTS, GLOBAL_MODEL_PATH, BATCH_SIZE, NUM_FEATURES
from torch.utils.data import DataLoader, TensorDataset
from utils import to_tensor, construct_autoencoder
import pandas as pd
import pickle
import time


/home/sharedrive/PythonCodes/.venv311_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
STRATEGY = "Anomaly_FLAccShield"

In [26]:
result_sources = {
    'components': [27, 29, 31, 33, 35, 37, 39, 41],
    'folds': [1, 2, 3, 4, 5],
    #'folds': [1, 2],
    'marker': ['o', '-', '^' 'x', '-o-'],
    'clients': [1, 2, 3, 4],
    'path': './results/original_{0}_fold_{1}'
}

## 1. Helper Functions

In [14]:
# Load the global model from the saved path
def load_model(model_path=GLOBAL_MODEL_PATH, input_size=NUM_FEATURES, num_classes=NUM_CLASSES):
    model = construct_autoencoder(input_size=input_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

In [15]:
# Run inference on a client's dataset
def run_inference(model, dataloader, device):
    all_preds = []
    all_labels = []

    # Get the total number of samples from the DataLoader
    total_samples = len(dataloader.dataset)
    # Start the timer before the loop
    start_time = time.time()
    
    with torch.no_grad():
        for batch in dataloader:
            features, labels = batch[0].to(device), batch[1].to(device)
            outputs = model(features)
            _, preds = torch.max(outputs.data, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # End the timer after the loop
    end_time = time.time()    
    # Calculate the total inference time
    total_inference_time = end_time - start_time    
    # Calculate average inference time per sample
    #inference_time_per_sample =  total_inference_time * 1000
    inference_time_per_sample =  total_inference_time * 1000000 / total_samples
    
    return np.array(all_preds), np.array(all_labels), f'{inference_time_per_sample:.4f} us'

In [16]:
def plot_confusion_matrix(y_true, y_pred, classes, title, ax):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues, ax=ax)  # Pass ax here directly
    ax.set_title(title)  # Optional: Set a title for each subplot


## 2. Performance/History of Global Model

In [17]:
# ## This will read pickle file and return the metrics
# def parse_history(history_path):
#     with open(history_path, 'rb') as file:
#         history = pickle.load(file)
#         # Extract distributed and centralized losses
#         loss_distributed = [loss for _, loss in history.losses_distributed]
#         loss_centralized = [loss for _, loss in history.losses_centralized]
        
#         # Extract accuracy for distributed and centralized evaluation
#         accuracy_distributed = [acc for _, acc in history.metrics_distributed['accuracy']]
#         accuracy_centralized = [acc for _, acc in history.metrics_centralized['accuracy']]

#         return loss_distributed, loss_centralized, accuracy_distributed, accuracy_centralized
    

In [18]:
##Parse the Training Time
def parse_training_time(time_path):
    # Open the text file and read the value
    with open(time_path, 'r') as file:
    # Read the content of the file and strip any extra spaces or newlines
        content = file.read().strip()
    # Convert the content to a float, round it, and cast it to an integer
    rounded_value = round(float(content))
    return int(rounded_value)


In [19]:
def plot_metrics(y_true, y_pred, classes, title, ax):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues, ax=ax)  # Pass ax here directly
    ax.set_title(title)  # Optional

### 2.1 Accuracy/Loss vs Round

In [20]:
# for component in result_sources.get('components'):    
#     loss_distributed = [] 
#     loss_centralized = [] 
#     accuracy_distributed =[] 
#     accuracy_centralized = []
   
#     for fold in result_sources.get('folds'):
#         history_path = result_sources.get('path').format(component, fold) + '/history.pkl'        
#         l_d, l_c, a_d, a_c = parse_history(history_path)        
#         loss_distributed.append(l_d)
#         loss_centralized.append(l_c)
#         accuracy_distributed.append(a_d)
#         accuracy_centralized.append(a_c)

#     history_plots = [
#         {
#             'type': 'distributed_loss',
#             'plot_name': 'Distributed Loss {}',
#             'x': 'Rounds',
#             'y': 'Loss',
#             'plot_position': [0, 0],
#             'data': loss_distributed,
#             'colors': ['red', 'brown', 'blue', 'purple', 'green']
#         },
#         {
#             'type': 'accuracy_distributed',
#             'plot_name': 'Distributed Accuracy {}',
#             'x': 'Rounds',
#             'y': 'Accuracy',
#             'plot_position': [0, 1],
#             'data': accuracy_distributed,
#             'colors': ['red', 'brown', 'blue', 'purple', 'green']
#         },
#          {
#             'type': 'centralized_loss',
#             'plot_name': 'Centralized Loss {}',
#             'x': 'Rounds',
#             'y': 'Loss',
#             'plot_position': [1, 0],
#             'data': loss_centralized,
#             'colors': ['red', 'brown', 'blue', 'purple', 'green']
#         },
#         {
#             'type': 'centralized_accuracy',
#             'plot_name': 'Centralized Accuracy {}',
#             'x': 'Rounds',
#             'y': 'Accuracy',
#             'plot_position': [1, 1],
#             'data': accuracy_centralized,
#             'colors': ['red', 'brown', 'blue', 'purple', 'green']
#         },     
      
#     ]

#     fig, ax = plt.subplots(2, 2, figsize=(16, 9))  # Adjust the figsize as needed
#     for plot in  history_plots:
#         position = plot.get('plot_position')
#         all_fold_data = plot.get('data')
#         #print(len(all_fold_data))
#         for i, data in enumerate(all_fold_data):
#             rounds = list(range(1, len(data)+1))
#             ax[position[0], position[1]].plot(rounds, data, label=f'Fold_{i+1}', marker='o', color=plot.get('colors')[i])
#             ax[position[0], position[1]].set_title(plot.get('plot_name').format(component))
#             ax[position[0], position[1]].set_xlabel(plot.get('x'))
#             ax[position[0], position[1]].set_ylabel(plot.get('y'))
#             ax[position[0], position[1]].legend()
#             ax[position[0], position[1]].grid(True)
            
            


### 2.2 Training Time

In [21]:
for component in result_sources.get('components'):
    training_time = []
    for fold in result_sources.get('folds'):
        ##Parsign the training time
        training_time_path = result_sources.get('path').format(component, fold) + '/training_time.txt'
        training_time.append(parse_training_time(training_time_path))
        #print(f'[Component {component} Fold {fold}]: {training_time} Seconds')

    training_time_to_string = ", ".join(map(str, training_time))
    print(f'{STRATEGY}, {component}, {training_time_to_string}')

Anomaly_FLAccShield, 27, 692, 761, 779, 672, 625
Anomaly_FLAccShield, 29, 529, 688, 705, 729, 575
Anomaly_FLAccShield, 31, 705, 736, 673, 548, 528
Anomaly_FLAccShield, 33, 502, 436, 489, 479, 487
Anomaly_FLAccShield, 35, 527, 478, 493, 533, 519
Anomaly_FLAccShield, 37, 540, 468, 548, 565, 556
Anomaly_FLAccShield, 39, 322, 427, 450, 472, 442
Anomaly_FLAccShield, 41, 481, 486, 488, 463, 455


## 3. Accumulate Results
- Accumulate all thre results and save in csv file
- It also stores values reauired for confusion matrix in a varialbe

In [24]:
# import torch
# import torch.nn as nn
# import numpy as np
# import pandas as pd
# import time
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
# from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
client_metrics = {
    'Strategy': [],
    'Component': [],
    'Fold': [],
    'Client': [],
    'Accuracy': [],
    'F1_Score': [],
    'Precision': [],
    'Recall': [],
    'ROC_AUC': [],
    'Confusion_Matrix': [],
    'CM_String': [],
    'Sample_Number': [],
    'Inference_Time_Per_Sample': []
}

def run_unsupervised_inference(model, dataloader, device):
    """Run inference on AutoEncoder model for anomaly detection."""
    reconstruction_errors = []
    all_labels = []
    
    total_samples = len(dataloader.dataset)
    start_time = time.time()
    
    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch[0].to(device), batch[1].to(device)
            outputs = model(inputs)
            
            # Compute per-sample reconstruction error (MSE loss)
            errors = torch.mean((outputs - inputs) ** 2, dim=1).cpu().numpy()
            reconstruction_errors.extend(errors)
            all_labels.extend(labels.cpu().numpy())
    
    end_time = time.time()
    inference_time_per_sample = (end_time - start_time) * 1000000 / total_samples  # Microseconds per sample
    
    return np.array(reconstruction_errors), np.array(all_labels), f'{inference_time_per_sample:.4f} us'

def accumulate_results(results, confusion_matrix_data, model_to_use):
    components = results.get('components')
    folds = results.get('folds')
    path = results.get('path')
    clients = results.get('clients')
    
    for component in components:  
        for fold in folds:
            global_model = path.format(component, fold) + '/' + model_to_use
            try:
                model = load_model(model_path=global_model, input_size=component)
                model.to(device)
            except:
                continue
            
            for client in clients:                                
                testset = get_evaluation_datasets_by_client(client, fold=fold, feature_count=component) 
                testloader = DataLoader(to_tensor(testset, "eval"), batch_size=BATCH_SIZE)
                
                # Running inference
                reconstruction_errors, labels, inference_time_per_sample = run_unsupervised_inference(model, testloader, device)
                
                # Compute anomaly detection threshold (95th percentile of benign samples)
                benign_errors = [err for err, lbl in zip(reconstruction_errors, labels) if lbl == 0]
                #threshold = np.percentile(benign_errors, 95)  
                threshold = 0.2
                
                # Predict anomalies (1 if reconstruction error > threshold, else 0)
                preds = (np.array(reconstruction_errors) > threshold).astype(int)
                
                # Compute evaluation metrics
                avg_loss = np.mean(reconstruction_errors)
                accuracy = accuracy_score(labels, preds)
                f1 = f1_score(labels, preds)
                precision = precision_score(labels, preds)
                recall = recall_score(labels, preds)
                roc_auc = roc_auc_score(labels, reconstruction_errors)
                cm = confusion_matrix(labels, preds)
                cm_str = f"[[{cm[0][0]} {cm[0][1]}] [{cm[1][0]} {cm[1][1]}]]"
                
                # Store results
                client_metrics['Strategy'].append(STRATEGY)
                client_metrics['Component'].append(component)
                client_metrics['Fold'].append(fold)
                client_metrics['Client'].append(client)
                client_metrics['Accuracy'].append(accuracy)
                client_metrics['F1_Score'].append(f1)
                client_metrics['Precision'].append(precision)
                client_metrics['Recall'].append(recall)
                client_metrics['ROC_AUC'].append(roc_auc)
                client_metrics['Confusion_Matrix'].append(cm)
                client_metrics['CM_String'].append(cm_str)
                client_metrics['Sample_Number'].append(len(labels))
                client_metrics['Inference_Time_Per_Sample'].append(inference_time_per_sample)

                # Save confusion matrix data
                key = f'{component}_{fold}_{client}'
                confusion_matrix_data[key] = {
                    'preds': preds,
                    'labels': labels
                }   
    
    # Convert results to DataFrame
    df = pd.DataFrame(client_metrics)
    print(df.to_string(index=False))
    return df, confusion_matrix_data   

confusion_matrix_data = {}
result_df, store_results_df = accumulate_results(result_sources, confusion_matrix_data, 'best_global_model.pth')


           Strategy  Component  Fold  Client  Accuracy  F1_Score  Precision   Recall  ROC_AUC                Confusion_Matrix                    CM_String  Sample_Number Inference_Time_Per_Sample
Anomaly_FLAccShield         27     3       1  0.738881  0.731576   0.780846 0.688155 0.853168 [[32176, 8389], [13545, 29890]] [[32176 8389] [13545 29890]]          84000                18.3675 us
Anomaly_FLAccShield         27     3       2  0.680464  0.673057   0.782973 0.590203 0.830236 [[29531, 7658], [19183, 27628]] [[29531 7658] [19183 27628]]          84000                17.2861 us
Anomaly_FLAccShield         27     3       3  0.663393  0.608937   0.726463 0.524143 0.838494 [[33711, 8289], [19986, 22014]] [[33711 8289] [19986 22014]]          84000                17.2766 us
Anomaly_FLAccShield         27     3       4  0.672012  0.625752   0.728477 0.548418 0.840838 [[33416, 8585], [18966, 23033]] [[33416 8585] [18966 23033]]          84000                18.4132 us
Anomaly_FLAccShield 

## 4. Confusion Matrix (per client per Fold)

In [ ]:
plots = [
    {
        'client_id': 1,
        'plot_name': 'Client 1',
        'plot_position': [0, 0]
    },
    {
        'client_id': 2,
        'plot_name': 'Client 2',
        'plot_position': [0, 1]
    },
    {
        'client_id': 3,
        'plot_name': 'Client 3',
        'plot_position': [1, 0]
    },
    {
        'client_id': 4,
        'plot_name': 'Client 4',
        'plot_position': [1, 1]
    }   
]

In [ ]:

# for component in result_sources.get('components'):
#     for fold in result_sources.get('folds'):
#         #print(f" Plots for Components: {component}, and Folds: {fold}")
#         fig, ax = plt.subplots(2, 2, figsize=(12, 10))  # Adjust the figsize as needed
#         for plot in plots:
#             client = plot.get('client_id')
#             key = f'{component}_{fold}_{client}'
#             data = confusion_matrix_data[key]
#             title = "{0} ({1}_{2})".format(plot['plot_name'], component, fold)
#             plot_confusion_matrix(data['labels'], data['preds'], data['classes'], title, ax = ax[plot['plot_position'][0], plot['plot_position'][1]])
#         plt.tight_layout()
#         plt.show()
#         print("---------------------------------------------------------------------------------------------------------------------------------")
#         print("---------------------------------------------------------------------------------------------------------------------------------")



## 5. Accumulate All the Training/Validaiton Metrics

In [ ]:
components =  result_sources.get('components')
folds =  result_sources.get('folds')
path =  result_sources.get('path')
#clients = results.get('clients')

for component in components:
    metric_dfs = []
    for fold in folds:
        metric_path = path.format(component, fold) + '/' + 'metrics.csv'
        metric_df = pd.read_csv(metric_path)
        metric_df['fold'] = fold
        metric_dfs.append(metric_df)
    #merging
    merged_df = pd.concat(metric_dfs, ignore_index=True)
    merged_df.to_csv(f"./results/accumulted_metrics_{component}.csv")

## 5. Accumulate All Local Metrics

In [ ]:
components =  result_sources.get('components')
folds =  result_sources.get('folds')
path =  result_sources.get('path')
clients = result_sources.get('clients')


for component in components:
    local_training_dfs = []
    for fold in folds:
        for client in clients:
            local_training_history_path = path.format(component, fold) + f"/local_train_history_{client}.csv"
            local_training_df = pd.read_csv(local_training_history_path)
            local_training_df['fold'] = fold
            local_training_dfs.append(local_training_df)
    #merging
    local_training_dfs_merged = pd.concat(local_training_dfs, ignore_index=True)
    local_training_dfs_merged.to_csv(f"./results/local_training_metrics_{component}.csv")
